don't care about v_s, we can't deal with that yet \
important: find k_A/k_B \
vuln is that finding k_X * i where i is the random number factor in v, is trivial by working backwards \
k_A\*v % n and k_B\*v % n are easily satisfied \
(vka * k_B) % n harder \

In [240]:
from Crypto.Util.number import getPrime, GCD, bytes_to_long, long_to_bytes, inverse
from Crypto.Cipher import AES
from hashlib import sha256

In [249]:
p = 10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
q = 11956676387836512151480744979869173960415735990945471431153245263360714040288733895951317727355037104240049869019766679351362643879028085294045007143623763
n = p*q
vka = 124641741967121300068241280971408306625050636261192655845274494695382484894973990899018981438824398885984003880665335336872849819983045790478166909381968949910717906136475842568208640203811766079825364974168541198988879036997489130022151352858776555178444457677074095521488219905950926757695656018450299948207
vkb = 6568897840127713147382345832798645667110237168011335640630440006583923102503659273104899584827637961921428677335180620421654712000512310008036693022785945317428066257236409339677041133038317088022368203160674699948914222030034711433252914821805540365972835274052062305301998463475108156010447054013166491083
vkakb = 114778245184091677576134046724609868204771151111446457870524843414356897479473739627212552495413311985409829523700919603502616667323311977056345059189257932050632105761365449853358722065048852091755612586569454771946427631498462394616623706064561443106503673008210435922340001958432623802886222040403262923652
c = bytes.fromhex('fef29e5ff72f28160027959474fc462e2a9e0b2d84b1508f7bd0e270bc98fac942e1402aa12db6e6a36fb380e7b53323')
pkb = (vkakb * inverse(vka//p , n)) % n
pka = (vkakb * inverse(vkb//p , n)) % n
j_A = pka//p
j_B = pkb//p
pj_r_modn = (vka * inverse(j_A, n)) % n
assert pj_r_modn == (vkb * inverse(j_B, n)) % n
j_r = pj_r_modn//p
v_s= (vkakb * inverse(j_A, n) * inverse(j_B, n)) % n # j_A and j_B can be multiplied by any multiple of q to get the same ans
cipher = AES.new(sha256(long_to_bytes(v_s)).digest(), AES.MODE_ECB)
m = cipher.decrypt(c)
m

b'crypto{1nv3rt1bl3_k3y_3xch4ng3_pr0t0c0l}\x08\x08\x08\x08\x08\x08\x08\x08'

In [243]:
inverse(vka//p, n)

5638118808468248798334318052164196330450126241678242396587162517665623937625431306489366574065490148498876950898488246398494891215436067342133590448011647822102649782709627274433784557171823191045504716065640237513640640412748758585735505147621241087090049363223271930913012438513576963277924016164787337262

In [151]:
pkb = (vkakb * inverse(q*vka // n , n)) % n
pka = (vkakb * inverse(q*vkb // n , n)) % n
# oops, this is if gcd(k_A, n)!=1, my bad really
j_A = pka//p
k_A = (j_A * inverse(p, q)) % q
i_A = (k_A*p - j_A)//q
k_A = ((i_A-3)*q+j_A)

j_B = pkb//p
k_B = (j_B * inverse(p, q)) % q
i_B = (k_B*p - j_B)//q
k_B = ((i_B-3)*q+j_B)

v_s = vkb * inverse(k_B, n)
cipher = AES.new(sha256(long_to_bytes(v_s)).digest(), AES.MODE_ECB)
m = cipher.decrypt(c)
m

b'~\x8b\xff\x15\xa0\xa9\x83\xb7M\xd6\x00E5}\xbd\xac\xcfH\x0fn\x82h\x99}H\x00\xec\xabX\xdb3\x8d\xec7\xb1\x1c\x02.\x92T\xa9\xcc\x8e\x0e}\xc2\xdb\x8e'

In [222]:
# j_Aj_r = vka//p
# j_Bj_r = vkb//p
# i_r = 1#GCD(j_Ai_r, j_Bi_r) # since result is 3, a prime, the only solutions to i_r are 1 and 3
j_A = pka//p
j_B = pkb//p
pj_r_modn = (vka * inverse(j_A, n)) % n
assert pj_r_modn == (vkb * inverse(j_B, n)) % n
j_r = pj_r_modn//p

In [223]:
p*j_B*j_r % n

6568897840127713147382345832798645667110237168011335640630440006583923102503659273104899584827637961921428677335180620421654712000512310008036693022785945317428066257236409339677041133038317088022368203160674699948914222030034711433252914821805540365972835274052062305301998463475108156010447054013166491083

In [238]:

AES.new(sha256(long_to_bytes((vkakb * inverse(3*q+j_A, n) * inverse(3*q+j_B, n)) % n)).digest(), AES.MODE_ECB).decrypt(c)

b'crypto{1nv3rt1bl3_k3y_3xch4ng3_pr0t0c0l}\x08\x08\x08\x08\x08\x08\x08\x08'

In [230]:
(vka * inverse(3*q+j_A, n)) % n

20874156146766564057942672668457062326147889691993938663991403860756979765642714212645458239678000453085625600572189661716538843500217234215652282314079009067289630661785411097593217597517876615856722749135735945217320827494378370904257343419854527927883822485663158720120367096738314111922498226725135680737

In [231]:
p*(3*q+j_r) % n

20874156146766564057942672668457062326147889691993938663991403860756979765642714212645458239678000453085625600572189661716538843500217234215652282314079009067289630661785411097593217597517876615856722749135735945217320827494378370904257343419854527927883822485663158720120367096738314111922498226725135680737

In [225]:
inverse(j_r*j_B, n)

100124768312046835973475097663195501494529597601343422589454230257632535630363738358751261713760357246032066010644089252678410030956404850032335335488161952855635958382455156073251938866769541659993470240809824025414428671794680547434736958884293202171315876195374158980733696509844231754885039215312177343770

In [217]:
(vkb * inverse(j_B, n)) % n

20874156146766564057942672668457062326147889691993938663991403860756979765642714212645458239678000453085625600572189661716538843500217234215652282314079009067289630661785411097593217597517876615856722749135735945217320827494378370904257343419854527927883822485663158720120367096738314111922498226725135680737

In [218]:
pj_r_modn % n

20874156146766564057942672668457062326147889691993938663991403860756979765642714212645458239678000453085625600572189661716538843500217234215652282314079009067289630661785411097593217597517876615856722749135735945217320827494378370904257343419854527927883822485663158720120367096738314111922498226725135680737

In [150]:
(vka//p) % q

11648825546348333563737888516661175025747799143080696127905803483286669957866521190417669223301528693034284833468311623325218575300897722560943664979777057

In [126]:
GCD(2*k_A, n)

10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951

In [118]:
j = pka//p
k = (j * inverse(p, q)) % q
i = (k*p - j)//q
k_A = (i*q+j)
GCD(k_A, n)

10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951

In [108]:
(7*q+pka//p) % p

9435167217250829794192614409328896785457903737738098127213667035812974939937448582409795114457223762738174505079617012774605601330724320320460689724631638

In [84]:
(vkakb * inverse(3*q+(pka//p), n)) % n

6568897840127713147382345832798645667110237168011335640630440006583923102503659273104899584827637961921428677335180620421654712000512310008036693022785945317428066257236409339677041133038317088022368203160674699948914222030034711433252914821805540365972835274052062305301998463475108156010447054013166491083

5678143837118796058091580139882173552685946733571815992274932757917187853454900823757290386606590959183181182074317657315855005859520003484526995857581055

In [ ]:
6568897840127713147382345832798645667110237168011335640630440006583923102503659273104899584827637961921428677335180620421654712000512310008036693022785945317428066257236409339677041133038317088022368203160674699948914222030034711433252914821805540365972835274052062305301998463475108156010447054013166491083
6568897840127713147382345832798645667110237168011335640630440006583923102503659273104899584827637961921428677335180620421654712000512310008036693022785945317428066257236409339677041133038317088022368203160674699948914222030034711433252914821805540365972835274052062305301998463475108156010447054013166491083

In [13]:
for i in range(10000000):
    if ((vka+i*n)%vkakb==0):
        print(i)

In [4]:
from collections import Counter
v_candidates = Counter([GCD(vka+i*n, vkb+i*n) for i in range(1000000,1000000+1000000)]).keys()


In [5]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
out = ""
keys = [sha256(long_to_bytes(GCD(vka, vkb)*i)).digest() for i in range(10000)]
for key in keys:
    cipher = AES.new(key, AES.MODE_ECB)
    out+=str(cipher.decrypt(c))+"\n"
open("cat.txt", 'w').write(out)

1416310

In [81]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
out = ""
keys = [sha256(long_to_bytes(GCD(vka-vkb, n)*i)).digest() for i in range(10000)]
for key in keys:
    cipher = AES.new(key, AES.MODE_ECB)
    out+=str(cipher.decrypt(c))+"\n"
open("cat2.txt", 'w').write(out)

1416258

In [ ]:
# a
# 3a
# b
# 2a
# 2b
# 4b
# 7a

In [102]:
GCD(vka-vkb, n)*890499

9528286447278255824174992139348570481855874313583041375218341154139989046061365354303638791736602741720746280472895454788785825229736325386118363240389914733549

In [98]:
GCD(vka-vkb, n)*4 # a

42799762592785644112682853722906237881708454758884811213570553831683085757811588128919353269286558398025135482343699228359766042318908052164543085350527804

In [96]:
GCD(vka, vkb)*4 # b

128399287778356932338048561168718713645125364276654433640711661495049257273434764386758059807859675194075406447031097685079298126956724156493629256051583412

In [15]:
GCD(vka-vkb, n)

10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951

In [55]:
count = Counter([GCD(vka+i*n, vkb+i*n) for i in range(0, 1*2**20, 1)])
count

Counter({10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951: 294680,
         42799762592785644112682853722906237881708454758884811213570553831683085757811588128919353269286558398025135482343699228359766042318908052164543085350527804: 147342,
         32099821944589233084512140292179678411281341069163608410177915373762314318358691096689514951964918798518851611757774421269824531739181039123407314012895853: 147341,
         21399881296392822056341426861453118940854227379442405606785276915841542878905794064459676634643279199012567741171849614179883021159454026082271542675263902: 147339,
         64199643889178466169024280584359356822562682138327216820355830747524628636717382193379029903929837597037703223515548842539649063478362078246814628025791706: 73671,
         12839928777835693233804856116871871364512536427665443364071166149504925727343476438675805980785967519407540644703109768507

In [53]:
gcds = [entry[0] for entry in count.most_common()]
a = GCD(vka-vkb, n)
b = GCD(vka, vkb)
coeffs = []
for g in gcds:
    if (g%a==0):
        coeffs.append(g//a)
coeffs.sort()
coeffs

[1,
 2,
 3,
 4,
 6,
 7,
 12,
 14,
 21,
 28,
 42,
 61,
 84,
 122,
 183,
 244,
 366,
 427,
 732,
 854,
 1281,
 1708,
 2562,
 5124,
 296833,
 593666,
 890499]

In [51]:
count2 = Counter([GCD(vka+i*n, vkb+i*n) for i in range(0, 1*2**18, 1)])
count2

Counter({10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951: 73668,
         32099821944589233084512140292179678411281341069163608410177915373762314318358691096689514951964918798518851611757774421269824531739181039123407314012895853: 36837,
         42799762592785644112682853722906237881708454758884811213570553831683085757811588128919353269286558398025135482343699228359766042318908052164543085350527804: 36836,
         21399881296392822056341426861453118940854227379442405606785276915841542878905794064459676634643279199012567741171849614179883021159454026082271542675263902: 36835,
         64199643889178466169024280584359356822562682138327216820355830747524628636717382193379029903929837597037703223515548842539649063478362078246814628025791706: 18418,
         128399287778356932338048561168718713645125364276654433640711661495049257273434764386758059807859675194075406447031097685079298

In [56]:
for i in range(0,len(c.keys())-1):
    print(list(c.keys())[i+1] - list(c.keys())[i])

10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
-32099821944589233084512140292179678411281341069163608410177915373762314318358691096689514951964918798518851611757774421269824531739181039123407314012895853
438697566576052852154999250659788938287511661278569314939098176774751629017568778321423371010187223579757638694022917090687601933768807534686566624842909991
-427997625927856441126828537229062378817084547588848112135705538316830857578115881289193532692865583980251354823436992283597660423189080521645430853505278040
106999406481964110281707134307265594704271136897212028033926384579207714394528970322298383173216395995062838705859248070899415105797270130411357713376319510
-53499703240982055140853567153632797352135568448606014016963192289603857197264485161149191586608197997531419352929624035449707552898635065205678856688159755
-106999406481964110281707134307265594704271136897212028033

In [22]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
keys = [sha256(long_to_bytes(v)).digest() for v in v_candidates]
for key in keys:
    cipher = AES.new(key, AES.MODE_ECB)
    print(cipher.decrypt(c))

b'\xcck\x11\xa0\x02L\xb89\xf5/\xaf\xb4G\x81\xcbY<F\xcf\x1cvs\x00!k\xd3s{\xcf=\x84(\x15\xa6\x9e^V\x9d\xd2\xf7)\x80d\xc0~}3\xb7'
b'd\x87\x0f\xb7yt=\xbb($\xd67\x08\x8a\xd3q\xfd\x99\xc9\x964\xea\xdaX\xaf\xfbJ!\x10\x97\xa4\xc8\xe3\xc7\xf7\xbb\xb2gi\xee\xaf\xdc\xcf\xbdT\x80\n\xcd'
b'\x9e\x03\xbf\xdd\xda\xd6\t\xcd\xe9)\x13\x18\x8akL\xee|\x8fAlW\x8c W\x15Jq\xc8\x87\x007`\xb2m\xb4\xa3E\x90\x8a\x93\xcfQ{\xd2\xbc\xb7\xe5\xf7'
b'3 \xa1x \xef|\x9c\x8cBe\xaf\xd8h-Tk\rl\r\x00&\x13\n\x95\\e\xb5\xd8\xf3\xfd\x9e\xec\xbd\xeeJe\x17\xae\xbe\xba\x7fb\x95\xf6\xf5\xca\xa8'
b'g\xb2\xfc\xfci\xcays.\x08)#\xafI\xac\xa2W\x99=\x1c\xb8\x89\xe6n\xdb\xf3\xa0\xd2\x04r\x95\xddG\xabgIa\xee\xa25\x98.\xd4u\xa2\xfe!\xfb'
b"y[\x87=\xb0\x1fl\x92gU\xff'4q\xccr1n&O\x7f\xdeR|\x8f<\x04\xe9\xbeMKdu\xbf\xaeJ\x06\xfbz\xfd$\x1b\xec\xbb\xc9\xe8]\x13"
b"\xe7\xe63p\xa5\xd1s\xcf\x9e\x1f\xde\x07\xff\xc8\x01\xa6\x84\x7f{\x12\xe1Z^\xb9\x1d\x98\xa3x9R9\x8ce%[\x8fAb\x86'\xf6\xf6p \x89\xdeR\xfa"
b'\xbf\xaa\xcdD\xce\x7f\xaf\xcc^\xfd\xe3o\x8c\t\

In [6]:
32099821944589233084512140292179678411281341069163608410177915373762314318358691096689514951964918798518851611757774421269824531739181039123407314012895853
42799762592785644112682853722906237881708454758884811213570553831683085757811588128919353269286558398025135482343699228359766042318908052164543085350527804
10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
449397507224249263183169964090515497757938774968290517742490815232672400457021675353653209327508863179263922564608841897777543444348534547727702396180541942
10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
42799762592785644112682853722906237881708454758884811213570553831683085757811588128919353269286558398025135482343699228359766042318908052164543085350527804
32099821944589233084512140292179678411281341069163608410177915373762314318358691096689514951964918798518851611757774421269824531739181039123407314012895853
21399881296392822056341426861453118940854227379442405606785276915841542878905794064459676634643279199012567741171849614179883021159454026082271542675263902
10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
128399287778356932338048561168718713645125364276654433640711661495049257273434764386758059807859675194075406447031097685079298126956724156493629256051583412
74899584537374877197194994015085916292989795828048419623748469205445400076170279225608868221251477196543987094101473649629590574058089091287950399363423657
21399881296392822056341426861453118940854227379442405606785276915841542878905794064459676634643279199012567741171849614179883021159454026082271542675263902
32099821944589233084512140292179678411281341069163608410177915373762314318358691096689514951964918798518851611757774421269824531739181039123407314012895853
42799762592785644112682853722906237881708454758884811213570553831683085757811588128919353269286558398025135482343699228359766042318908052164543085350527804
10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
64199643889178466169024280584359356822562682138327216820355830747524628636717382193379029903929837597037703223515548842539649063478362078246814628025791706
10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
299598338149499508788779976060343665171959183312193678494993876821781600304681116902435472885005908786175948376405894598518362296232356365151801597453694628
32099821944589233084512140292179678411281341069163608410177915373762314318358691096689514951964918798518851611757774421269824531739181039123407314012895853
21399881296392822056341426861453118940854227379442405606785276915841542878905794064459676634643279199012567741171849614179883021159454026082271542675263902
10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
128399287778356932338048561168718713645125364276654433640711661495049257273434764386758059807859675194075406447031097685079298126956724156493629256051583412
10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
21399881296392822056341426861453118940854227379442405606785276915841542878905794064459676634643279199012567741171849614179883021159454026082271542675263902
224698753612124631591584982045257748878969387484145258871245407616336200228510837676826604663754431589631961282304420948888771722174267273863851198090270971
42799762592785644112682853722906237881708454758884811213570553831683085757811588128919353269286558398025135482343699228359766042318908052164543085350527804
10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
64199643889178466169024280584359356822562682138327216820355830747524628636717382193379029903929837597037703223515548842539649063478362078246814628025791706
10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
42799762592785644112682853722906237881708454758884811213570553831683085757811588128919353269286558398025135482343699228359766042318908052164543085350527804

ValueError: base is not invertible for the given modulus

In [2]:
p = 10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
q = 11956676387836512151480744979869173960415735990945471431153245263360714040288733895951317727355037104240049869019766679351362643879028085294045007143623763
n = p*q
i1 = 3
v = (p*i1) % n # (p * 0...n) % n # given (p*(n-i1)) % n, approximately n - i1p
k_A_times_i1 = 11648825546348333563737888516661175025747799143080696127905803483286669957866521190417669223301528693034284833468311623325218575300897722560943664979777057
k_B_times_i1 = 613919091339536621646381800904577171252124424439804898925544813060683405335756717981327665426887883526165978968616732696072978713244164666096967126321733
k_A = k_A_times_i1//i1 # 0...n coprime n
k_B = k_B_times_i1//i1 # 0...n coprime n
# k_A*v % n
# k_Ai1p % n
vka = 124641741967121300068241280971408306625050636261192655845274494695382484894973990899018981438824398885984003880665335336872849819983045790478166909381968949910717906136475842568208640203811766079825364974168541198988879036997489130022151352858776555178444457677074095521488219905950926757695656018450299948207
# (vka * k_B) % n
vkakb = 114778245184091677576134046724609868204771151111446457870524843414356897479473739627212552495413311985409829523700919603502616667323311977056345059189257932050632105761365449853358722065048852091755612586569454771946427631498462394616623706064561443106503673008210435922340001958432623802886222040403262923652
# (vkakb * inverse(k_A, n)) % n  /   k_B*v % n
# k_Bi1p % n
vkb = 6568897840127713147382345832798645667110237168011335640630440006583923102503659273104899584827637961921428677335180620421654712000512310008036693022785945317428066257236409339677041133038317088022368203160674699948914222030034711433252914821805540365972835274052062305301998463475108156010447054013166491083
# v_s = (vkb * inverse(k_B, n)) % n
v_s = None

In [ ]:
p = 10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951
q = 11956676387836512151480744979869173960415735990945471431153245263360714040288733895951317727355037104240049869019766679351362643879028085294045007143623763
n = p*q
i1 = None
v = (p*i1) % n # (p * 0...n) % n # given (p*(n-i1)) % n, approximately n - i1p
k_A_times_i1 = None
k_B_times_i1 = None
k_A = k_A_times_i1//i1 # 0...n coprime n
k_B = k_B_times_i1//i1 # 0...n coprime n
# k_A*v % n
# k_Ai1p % n
vka = 124641741967121300068241280971408306625050636261192655845274494695382484894973990899018981438824398885984003880665335336872849819983045790478166909381968949910717906136475842568208640203811766079825364974168541198988879036997489130022151352858776555178444457677074095521488219905950926757695656018450299948207
# (vka * k_B) % n
vkakb = 114778245184091677576134046724609868204771151111446457870524843414356897479473739627212552495413311985409829523700919603502616667323311977056345059189257932050632105761365449853358722065048852091755612586569454771946427631498462394616623706064561443106503673008210435922340001958432623802886222040403262923652
# (vkakb * inverse(k_A, n)) % n  /   k_B*v % n
# k_Bi1p % n
vkb = 6568897840127713147382345832798645667110237168011335640630440006583923102503659273104899584827637961921428677335180620421654712000512310008036693022785945317428066257236409339677041133038317088022368203160674699948914222030034711433252914821805540365972835274052062305301998463475108156010447054013166491083
# v_s = (vkb * inverse(k_B, n)) % n
v_s = None

In [68]:
(vkakb*1*n) // vka

117811562081479308718504792849888618971867756095087266069004631049441940836222296947556031823566497811805594890986091853083569249970107435264373166778289259542206525204086977012463467332190312302067354366641669122116808159976864894730531960043398372380600468373918915890011024969626374595820257179320580826081

In [61]:
(vkakb*n*2//vka)>n

True

2093994473898975360085519538488342320709776208326450611359844430850121144873927419110476152213787122185501846591341281492610942285339646949513153369586450586974161273112035436113385276595954408585833972984433305339993501383681283534841135416825777387257366779577938439736936244722101933939233412329502739221744272444043209885342568146990664434017261592113776550714472855579184428908950133393973236924261755961818667254517398912136884626725418091330912386659344581886158664334011221182591245755926060513397660008082368419870202302984924422347811405372787384386513828729901318103896216676888294738707677499181660893607820876832547031106804308294473586812462704282978721666065178992568246273946925682026881515287253612706750102988541109977790141843390539956982943624865405114189624655255557222148863697508762199543204105343886070879805882488516717088622815991962939723621623483473116267296683437162783657666995373462312623203397

In [37]:
p**2 % n

114488729874925832592117156733168368486498819120340197143805998981782512232179289707922522929707916721615966968232486138714166991190361353383293117619779304748676851098952931246823816049009809086734049485945710888795285099547973609338001637249069457949125556959301881256793686096942576726668351233776336066401

In [35]:
inverse(100, 7)

4

In [33]:
400%7

1

In [15]:
(vka*vkb)//n

6399767166842082183934116052006131571563455081219350292672535191941484136084079327699003811420469976898006075873749467346599718513086117825537052167318106454761568045082358922994118665432159604249538305354828727450046161649377160166431154542548090235457053752789368295738568780286840872791586216362822239603

In [220]:
from cryptolib import gcd
gcd(((vka+1*n)//p), ((vkb+2*n)//p))

(559,
 -12111203819047889220357895988184397111895017549860741564730262747531773381759366535238638676758688989119971042164446019482449277271253700874557446025560,
 11656047469361849204163599359996206988319833362493619094439122483489803594762397103027506814997348160209407944770741382190916119365765365243163966779501)

In [256]:
(vka*vkb//p//(10**101)) % n

120751431219134299638617509230038262208202637846508114912934312421233400502388476717749050519491630284289695012647953361492638950940502504178596115971027319055744022102789999257361197685637810010511843797473026193048496435611889890396059939884593790441029426551321860417842424481496677993535163381546020591102

In [223]:
vkakb

114778245184091677576134046724609868204771151111446457870524843414356897479473739627212552495413311985409829523700919603502616667323311977056345059189257932050632105761365449853358722065048852091755612586569454771946427631498462394616623706064561443106503673008210435922340001958432623802886222040403262923652

In [177]:
k_B*v % n

6568897840127713147382345832798645667110237168011335640630440006583923102503659273104899584827637961921428677335180620421654712000512310008036693022785945317428066257236409339677041133038317088022368203160674699948914222030034711433252914821805540365972835274052062305301998463475108156010447054013166491083

In [164]:
(p*(n-10**100)) % n

127935727699542132251014217893285758757968248710138489013984823432315762919961426040644070746281963618173019000815212340674798550606036730823956524642157818070128222469236416734399250551895455730439851096219812279119355991686749746498933168649629280437193164737435568227222775657559618986214668636901311651613

In [153]:
from Crypto.Util.number import getPrime, GCD, bytes_to_long, long_to_bytes, inverse
k_A*i1*p % n

124641741967121300068241280971408306625050636261192655845274494695382484894973990899018981438824398885984003880665335336872849819983045790478166909381968949910717906136475842568208640203811766079825364974168541198988879036997489130022151352858776555178444457677074095521488219905950926757695656018450299948207

In [138]:
k_A_times_i1 = (n-307850841488178587742856463207998934667936847864775303247441780074044082422212705533648504053508411205765035551455056026144068578130362733101342163846706)
k_B_times_i1 = (n//2-5364419102578719454093990689030009808955743571032930816651077818619673614808610229994331198250630668593858955541266606979608343226269877980925536445490148)
v = p*(n-3) % n

In [139]:
from cryptolib import gcd
gcd(k_A_times_i1, k_B_times_i1)

(3,
 4540840091256886020275783014306083905413679579783368923922689291187200361863830139244915140781201607247850768170972180933092953276640138204742397313957931234571641276196795496174914018376935149514994822267573743875232192337451215760484663725822211968014073399377547663144253429123927185113706304462579170421,
 -9081680182513772040551566028612167810827359159566737847845378582374400723727660278489830281562403214495701536341944361866185906553280276409484794627915863208890311194184530709676643482960916754977494746794856036377853699744423621400301692826477644221193810799953860889388148007179266581355709927013015383018)

In [143]:
v = (p*(3)) % n
k_A = k_A_times_i1//3
(k_A*v) % n

124641741967121300068241280971408306625050636261192655845274494695382484894973990899018981438824398885984003880665335336872849819983045790478166909381968949910717906136475842568208640203811766079825364974168541198988879036997489130022151352858776555178444457677074095521488219905950926757695656018450299948207

In [75]:
(p*(n-307850841488178587742856463207998934667936847864775303247441780074044082422212705533648504053508411205765035551455056026144068578130362733101342163846706)) % n

124641741967121300068241280971408306625050636261192655845274494695382484894973990899018981438824398885984003880665335336872849819983045790478166909381968949910717906136475842568208640203811766079825364974168541198988879036997489130022151352858776555178444457677074095521488219905950926757695656018450299948207

In [77]:
((-(vkb - n))//p)

11342757296496975529834363178964596789163611566505666532227700450300030634952977177969990061928149220713883890051149946655289665165783920627948040017302030

In [ ]:
n - i*p = vka

In [ ]:
(p*(n//2 + i)) % n = vkb

In [137]:
p*(n//2-5364419102578719454093990689030009808955743571032930816651077818619673614808610229994331198250630668593858955541266606979608343226269877980925536445490148) % n

6568897840127713147382345832798645667110237168011335640630440006583923102503659273104899584827637961921428677335180620421654712000512310008036693022785945317428066257236409339677041133038317088022368203160674699948914222030034711433252914821805540365972835274052062305301998463475108156010447054013166491083

In [136]:
((vkb+n*5349970324098205514085356715363279735213556844860601401696319228960385719726448516114919158660819799753141935292962403544970755289863506520567885668815975)//p) - n//2

-5364419102578719454093990689030009808955743571032930816651077818619673614808610229994331198250630668593858955541266606979608343226269877980925536445490148

In [111]:
((p*(n//2 + 10**152+6*10**151)) - ((p*(n//2 + 10**152+6*10**151)) % n))//n

5349970324098205514085356715363279735213556844860601401696319228960385719726448516114919158660819799753141935292962403544970755289863506520567885668815975

In [110]:
(p*(n//2 + 10**152+6*10**151)) % n# approximately n - i*p

65679854353482491890014423095559128894252462545424637009034937110407260031146743699111606856048012593613528936664546429075647114260259848647751572343298013376513140066036729731550462811303057857341937375668259443240449035457655146800950469405655979398796829226782491151207842858024519629600813750564987009831

In [26]:
n - p

127935727699542132251014217893285758757968248710138489120984229914279873201668560347909665450553100515385047034741596919882512945135007053122339697858553802433026280132073459463100925622606115714683874751336518886480898070915310293601900938811311958797593658453564982302415685716049039259201627501129974019662

In [6]:
n - (p*(n-1)) % n

10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951

In [ ]:
127935727699542132251014217893285758757968248710138489120984229914279873201668560347909665450553100515385047034741596919882512945135007053122339697858553802433026280132073459463100925622606115714683874751336518886480898070915310293601900938811311958797593658453564982302415685716049039259201627501129974019662
127935727699542132251014217893285758757968248710138489120984229914279873201668560347909665450553100515385047034741596919882512945135007053122339697858553791733085631935662431292387494896046645287570185030133715493842440150143870840704868708972994637157994152169694396377608595774538459532188586365358636387711